In [1]:
import numpy as np
import pandas as pd
import re
import gzip
import json
import shutil
#from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [12]:
import json
import pandas as pd

# Función para cargar y corregir cada línea del archivo JSON original
def cargar_y_corregir_linea(linea):
    # Cambia comillas simples por comillas dobles
    linea_corregida = linea.replace("'", '"')
    return json.loads(linea_corregida)

# Lista para almacenar los datos desanidados
data_list = []

# Lee el archivo JSON original, corrige y procesa cada línea
with open(r'C:\Users\gaba_\Desktop\proyectos Henrry\pi1-steam\pi1-venv\datasets\UserItems.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            # Obtén datos del usuario
            user_id = entrada['user_id']
            items_count = entrada['items_count']
            steam_id = entrada['steam_id']
            user_url = entrada['user_url']
            # Itera sobre cada ítem
            for item in entrada['items']:
                # Crea un diccionario para cada ítem y agrégalo a la lista
                item_dict = {
                    'user_id': user_id,
                    'items_count': items_count,
                    'steam_id': steam_id,
                    'user_url': user_url,
                    'item_id': item['item_id'],
                    'item_name': item['item_name'],
                    'playtime_forever': item['playtime_forever'],
                    'playtime_2weeks': item['playtime_2weeks']
                }
                data_list.append(item_dict)
        except json.JSONDecodeError:
            None

# Crea el DataFrame
df = pd.DataFrame(data_list)

# Guarda el DataFrame en un nuevo archivo CSV
df.to_csv('UsersItems.csv', index=False)


print("Se han guardado correctamente 'UsersItems.csv'")

Se han guardado correctamente 'UsersItems.csv'


In [14]:
# Ruta del archivo .json.gz
archivo_json_gz = '../datasets/data comprimida/user_reviews.json.gz'
# Ruta para guardar el archivo .json resultante
archivo_json = '../datasets/data comprimida/UserReviews.json'

# Descomprimir el archivo .json.gz
with gzip.open(archivo_json_gz, 'rb') as f_in:
    with open(archivo_json, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Archivo .json descomprimido y guardado como:", archivo_json)

Archivo .json descomprimido y guardado como: ../datasets/data comprimida/UserReviews.json


In [3]:
# Función para corregir y cargar cada línea del archivo 'australian_user_reviews.json' original
def cargar_y_corregir_linea(linea):
    # Reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return json.loads(linea_corregida)

# Lista para almacenar los datos desanidados
data_list = []

# Expresión regular para extraer números de la columna 'funny'
numero_regex = re.compile(r'\d+')

# Lee el archivo 'australian_user_reviews.json' original, corrige y procesa cada línea
with open('../datasets/UserReviews.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            user_id = entrada['user_id']
            user_url = entrada['user_url']
            # Iteración sobre cada reseña
            for reseña in entrada['reviews']:
                # Extrae el número de la columna 'funny'
                funny_valor = re.search(numero_regex, reseña.get('funny', ''))
                if funny_valor:
                    funny = int(funny_valor.group())
                else:
                    funny = None
                
                # Elimina 'Posted' de la columna 'posted'
                posted = reseña.get('posted', '').replace('Posted ', '', 1)

                reseña_dict = {
                    'user_id': user_id,
                    'user_url': user_url,
                    'funny': funny,
                    'posted': posted,
                    'item_id': int(reseña.get('item_id', '')),  # Convierte a entero
                    'helpful': reseña.get('helpful', ''),
                    'recommend': reseña.get('recommend', ''),
                    'review': reseña.get('review', '')  # Mantiene el texto original de la review
                }
                data_list.append(reseña_dict)
        except json.JSONDecodeError as e:
            None

# Crea el DataFrame
df = pd.DataFrame(data_list)

 

# Función para analizar el sentimiento de la reseña
def analyze_sentiment(review):
    if pd.isna(review) or not isinstance(review, str):
        return 1  # Sentimiento neutral para reseñas faltantes o no legibles
    else:
        # Se emplea TextBlob para calcular la polaridad del sentimiento
        polarity = TextBlob(review).sentiment.polarity
        if polarity < 0:
            return 0  # Sentimiento negativo
        elif polarity == 0:
            return 1  # Sentimiento neutral
        else:
            return 2  # Sentimiento positivo

# Aplica la función analyze_sentiment a cada reseña y crea la nueva columna 'sentiment_analysis'
df['sentiment_analysis'] = df['review'].apply(analyze_sentiment)

# Convierte la columna 'posted' a tipo datetime
df['posted'] = pd.to_datetime(df['posted'], errors='coerce')

# Elimina la columna original 'review'
df.drop(columns=['review'], inplace=True)

# Guarda el DataFrame modificado en un nuevo archivo CSV
df.to_csv('UserReviews.csv', index=False)



print("El análisis de sentimiento ha sido  guardado en 'UserReviews.csv'.")

C:\Users\gaba_\AppData\Local\Temp\ipykernel_9516\1595959858.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['posted'] = pd.to_datetime(df['posted'], errors='coerce')


El análisis de sentimiento ha sido  guardado en 'UserReviews.csv'.


In [19]:
data = pd.read_json("datasets/nuevo.json")

In [2]:
df2 = pd.read_csv('../datasets/UserReviews.csv')
df3 = pd.read_csv('../datasets/UsersItems.csv')
df4 = pd.read_csv('../datasets/datos.csv')

In [3]:
pd.options.mode.copy_on_write = True

In [6]:
def recomendar_juegos(df_datos, df_reviews, id_producto):
    # Selecciona y renombra columnas necesarias
    df_auc = df_datos[['genres', 'tags', 'specs', 'id']]
    df_auc2 = df_reviews[['item_id', 'recommend']].astype({'item_id': 'float'}).rename(columns={'item_id': 'id'})
    
    # Combina dataframes en base al ID
    df_auc3 = pd.merge(df_auc2, df_auc, on='id').reset_index(drop=True)
    df_auc3 = df_auc3[df_auc3['recommend'] != False]
    
    # Combina columnas de texto y vectoriza
    df_auc3['combined'] = df_auc3[['genres', 'tags']].fillna('').agg(' '.join, axis=1)
    
    # Configura el vectorizador TF-IDF
    vec = TfidfVectorizer(max_features=1000)  # Limita el número de características para mejorar el rendimiento
    vec_matrix = vec.fit_transform(df_auc3['combined'])
    
    # Calcula la similitud del coseno
    coseno = cosine_similarity(vec_matrix, dense_output=False)
    
    # Busca el juego en el DataFrame
    juego_en_data = df_datos[df_datos['id'] == id_producto]
    if not juego_en_data.empty:
        juego_indice = juego_en_data.index[0]
        # Obtiene los juegos similares
        juegos_similares = coseno[juego_indice].toarray().flatten()
        # Ordena de mayor a menor
        juegos_mas_similares = np.argsort(-juegos_similares)
        # Obtiene los 6 primeros
        top_5_juegos = df_datos.loc[juegos_mas_similares[1:6], 'app_name']
        # Convierte a lista
        top_5_juegos_mostrar = top_5_juegos.tolist()
        # Nombre del juego original
        nombre_del_juego = df_datos.loc[juego_indice, 'app_name']
        
        return 'Los 5 juegos recomendados para el id {} ({}) son: {}'.format(id_producto, nombre_del_juego, top_5_juegos_mostrar)
    else:
        return 'El juego no está en la base de datos'

In [7]:
print(recomendar_juegos(df4,df2,752360.0))

Los 5 juegos recomendados para el id 752360.0 (fallen mage) son: ['ghostbusters: sanctum of slime challenge pack dlc', 'blue reflection - bath towels set c (lime, fumio, chihiro)', 'the 9th day:第九日', 'crazy machines 2: invaders from space, 2nd wave dlc', 'labyrinth - starter pack']


"Los 5 juegos recomendados para el id 752360.0 (fallen mage) son: ['ghostbusters: sanctum of slime challenge pack dlc', 'blue reflection - bath towels set c (lime, fumio, chihiro)', 'the 9th day:第九日', 'crazy machines 2: invaders from space, 2nd wave dlc', 'labyrinth - starter pack']"

In [12]:
df_auc = df4[['genres','tags','specs','id']]
df_auc2= df2[['item_id','recommend']]
df_auc2['item_id'] =df_auc2['item_id'].astype(float)
df_auc2.rename(columns={'item_id':'id'},inplace=True)
df_auc3=pd.merge(df_auc2,df_auc,on='id').reset_index(drop=True)
df_auc3=df_auc3[(df_auc3['recommend']!=False)]
#transformo las columnas a srt para poder tabajarlas
#generos=df_auc3['genres'].astype(str)
#tags=df_auc3['tags'].astype(str)
#specs=df_auc3['specs'].astype(str)
vec=TfidfVectorizer()
vec_matrix1= vec.fit_transform(df_auc3['genres'])
#vec_matrix2= vec.fit_transform(df_auc3['tags'])
#vec_matrix3= vec.fit_transform(specs)

matrix_completa=np.column_stack([vec_matrix1.toarray()])
#calculo la similituid del coseno
coseno=cosine_similarity(matrix_completa)
#coseno=linear_kernel(matrix_completa,matrix_completa) 
id_producto=304930.0
#buaca el juego en el dataFrame
juego_en_data= df4[df4['id']== id_producto]
if not juego_en_data.empty:
    juego_indice=juego_en_data.index[0]
    #obtengo los similares
    juegos_similares=coseno[juego_indice]
    #los ordeno de mayor a menor
    juegos_mas_similares=np.argsort(-juegos_similares)
    #obtengo los 6 primeros
    top_5_juegos=df4.loc[juegos_mas_similares[0:6],'app_name']
    #los combierto en lista 
    top_5_juegos_mostrar=top_5_juegos.to_numpy().tolist()
    #tomo quito el primer valor para guardarlo en una variable para mostrar el nombre del juego que ingrese por id
    nombre_del_juego= top_5_juegos_mostrar.pop(0)
    a= ('los 5 juegos recomendados para el id {} ({}) son: {}'.format(id_producto,nombre_del_juego,top_5_juegos_mostrar) )
    #return print(f'los 5 juegos recomendados para el id {id_producto} ({nombre_del_juego}) son: {top_5_juegos_mostrar}' )
else:
    a='el juego no esta en la base de datos'

In [215]:
vec=TfidfVectorizer(min_df=5,max_df=0.98)
vec_matrix1= vec.fit_transform(df_auc3['genres'])


matrix_completa=np.column_stack(vec_matrix1.toarray())
#calculo la similituid del coseno


In [216]:
coseno2= linear_kernel(matrix_completa,matrix_completa)

In [217]:
coseno=cosine_similarity(matrix_completa)

In [9]:
print(a)

los 5 juegos recomendados para el id 304930.0 (crystalline) son: ['foresight ost', 'fantasy grounds - top down tokens - steampunk', 'beats fever - big apple', 'dead or alive 5 last round: core fighters add "dead or alive 3 music"', 'press x to not die - soundtrack']


In [ ]:
#paso todos precios a float, los que son F2P pasan a NaN
df4.loc[:,'price']=pd.to_numeric(df4['price'], errors='coerce')
#paso los NaN a 0
df4.loc[:,'price']=df4['price'].fillna(0)
#paso todo a minusculas
df4.loc[:,'developer'] = df4['developer'].str.lower()
#paso la columna a a formato de fecha
df4['release_date'] = pd.to_datetime(df4['release_date'],errors='coerce')



#transformo las columnas a srt para poder tabajarlas
generos=df4['genres'].astype(str)
tags=df4['tags'].astype(str)
specs=df4['specs'].astype(str)


vec=TfidfVectorizer()

#creo matriz
vec_matrix1= vec.fit_transform(generos)
vec_matrix2= vec.fit_transform(tags)
vec_matrix3= vec.fit_transform(specs)

#uno las matrises
matrix_completa=np.column_stack([vec_matrix1.toarray(),vec_matrix3.toarray(),vec_matrix2.toarray()])

#calculo la similituid del coseno
coseno=cosine_similarity(matrix_completa)
id_producto=248820.0
#buaca el juego en el dataFrame
juego_en_data= df4[df4['id']== id_producto]

if not juego_en_data.empty:
    juego_indice=juego_en_data.index[0]
    #obtengo los similares
    juegos_similares= coseno[juego_indice]
    #los ordeno de mayor a menor
    juegos_mas_similares=np.argsort(-juegos_similares)
    #obtengo los 6 primeros
    top_5_juegos=df4.loc[juegos_mas_similares[0:6],'app_name']
    #los combierto en lista 
    top_5_juegos_mostrar=top_5_juegos.to_numpy().tolist()
    #tomo quito el primer valor para guardarlo en una variable para mostrar el nombre del juego que ingrese por id
    nombre_del_juego= top_5_juegos_mostrar.pop(0)
    a= (f'los 5 juegos recomendados para el id {id_producto} ({nombre_del_juego}) son: {top_5_juegos_mostrar}' )
    #return print(f'los 5 juegos recomendados para el id {id_producto} ({nombre_del_juego}) son: {top_5_juegos_mostrar}' )
else:
    a='el juego no esta en la base de datos'

In [30]:
developer = 'Valve'
df_ra=df2[['item_id','sentiment_analysis']]
df_ra2=df4[['id','developer']]
df_ra2['id']=df_ra2['id'].astype(int)
df_ra.rename(columns={'item_id':'id'}, inplace=True)
df_ra3=pd.merge(df_ra,df_ra2,on='id')
df_ra3 = df_ra3.drop(columns='id')
df_ra3 = df_ra3[(df_ra3['sentiment_analysis']!=1)]
df_positivos = df_ra3[df_ra3['sentiment_analysis'] == 2].groupby('developer').size().reset_index(name='Positivos')
df_negativos = df_ra3[df_ra3['sentiment_analysis'] == 0].groupby('developer').size().reset_index(name='Negativos')

# Combinar los DataFrames
df_final = pd.merge(df_positivos, df_negativos, on='developer', how='outer').fillna(0)
df_final['Positivos'] = df_final['Positivos'].astype(int)
df_final['Negativos'] = df_final['Negativos'].astype(int)
df_salida=df_final[df_final['developer']==developer].reset_index()
dic_salida = {developer : [f'Negative = {df_salida['Negativos'].loc[0]}' , f'Positive = {df_salida["Positivos"].loc[0]}' ]}

C:\Users\gaba_\AppData\Local\Temp\ipykernel_11668\3363274597.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ra2['id']=df_ra2['id'].astype(int)
C:\Users\gaba_\AppData\Local\Temp\ipykernel_11668\3363274597.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ra.rename(columns={'item_id':'id'}, inplace=True)


In [32]:
dic_salida

{'Valve': ['Negative = 882', 'Positive = 3268']}

In [28]:
df_salida

,developer,Positivos,Negativos
1116,Valve,3268,882


In [18]:
df_final

,developer,Positivos,Negativos
0,11 bit studios,16,4
1,17-BIT,0,1
2,1C Company,0,1
3,1C: Maddox Games,1,0
4,22cans,1,0
...,...,...,...
1196,tinyBuild,2,0
1197,tobyfox,32,13
1198,△○□× (Miwashiba),3,0
1199,"インレ,Inre",0,1


In [34]:
def developer_reviews_analysis(desarrolador):
    df_ra=df2[['item_id','sentiment_analysis']]
    df_ra2=df4[['id','developer']]
    df_ra2['id']=df_ra2['id'].astype(int)
    df_ra.rename(columns={'item_id':'id'}, inplace=True)
    df_ra3=pd.merge(df_ra,df_ra2,on='id')
    df_ra3 = df_ra3.drop(columns='id')
    df_ra3 = df_ra3[(df_ra3['sentiment_analysis']!=1)]
    df_positivos = df_ra3[df_ra3['sentiment_analysis'] == 2].groupby('developer').size().reset_index(name='Positivos')
    df_negativos = df_ra3[df_ra3['sentiment_analysis'] == 0].groupby('developer').size().reset_index(name='Negativos')

    # Combinar los DataFrames
    df_final = pd.merge(df_positivos, df_negativos, on='developer', how='outer').fillna(0)
    df_final['Positivos'] = df_final['Positivos'].astype(int)
    df_final['Negativos'] = df_final['Negativos'].astype(int)
    df_salida=df_final[df_final['developer']==desarrolador].reset_index()
    dic_salida = {developer : [f'Negative = {df_salida['Negativos'].loc[0]}' , f'Positive = {df_salida["Positivos"].loc[0]}' ]}
    return dic_salida

In [35]:
developer_reviews_analysis('Valve')

C:\Users\gaba_\AppData\Local\Temp\ipykernel_11668\2081564235.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ra2['id']=df_ra2['id'].astype(int)
C:\Users\gaba_\AppData\Local\Temp\ipykernel_11668\2081564235.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ra.rename(columns={'item_id':'id'}, inplace=True)


{'Valve': ['Negative = 882', 'Positive = 3268']}

In [4]:
df4['release_date'] = pd.to_datetime(df4['release_date'], errors='coerce')


In [50]:
#guardamos el genero
genero='Adventure'
lista_jugos=[]
lista_ano=[]
val= 0
for i ,j , k in zip(df4['genres'],df4['release_date'].dt.year,df4['id']):
    if genero in i: 
        try:
            lista_ano.append(int(j))
            lista_jugos.append(int(k))
        except Exception as e:
            val+=1
df= pd.DataFrame({'item_id':lista_jugos,'año':lista_ano})
lista_usuarios=[]
lista_itesm=[]
lista_horas=[]
for i,j,k in zip(df3['user_id'],df3['item_id'],df3['playtime_forever']):
    if j in lista_jugos:
        lista_usuarios.append(i)
        lista_itesm.append(j)
        lista_horas.append(k)
new_df = pd.DataFrame({'user_id':lista_usuarios,'item_id':lista_itesm,'playtime_forever':lista_horas})
new_df = pd.merge(df, new_df, on='item_id')
new_df=new_df.sort_values(by='playtime_forever', ascending=False)
new_df.reset_index(drop=True, inplace=True)
new_df['playtime_forever'] = new_df['playtime_forever'].apply(lambda x: x / 60)
new_df['playtime_forever'] = new_df['playtime_forever'].round().astype(int)
new_df2 = new_df
new_df2=new_df2.drop(columns=['item_id'])
filtro_usuario=new_df2.groupby('user_id',)[['playtime_forever']].sum().reset_index()
new_df3 =new_df2.groupby(['año','user_id']).agg({'playtime_forever': 'sum'}).reset_index()
filtro_usuario=filtro_usuario.sort_values(by='playtime_forever', ascending=False)
filtro_usuario.reset_index(drop=True, inplace=True)
usuario=filtro_usuario['user_id'][0]
new_filtrado=new_df3[new_df3['user_id']==usuario]
new_filtrado = new_filtrado.sort_values(by='año', ascending=False)
new_filtrado.reset_index(drop=True, inplace=True)
lista_aux=[]
for i,j in zip(new_filtrado['año'],new_filtrado['playtime_forever']):
    dicc_aux={'Año':i,'Horas':j}
    lista_aux.append(dicc_aux)
diccionario_salida={f'Usuario con más horas jugadas para Género {genero}':usuario,
                        'Horas jugadas':lista_aux}


In [7]:
#df para trabajar y unir con el otro
ano=2000
df_auxi1= df4[['release_date','id','developer']]
df_auxi1['id'] = df_auxi1['id'].astype(int)
df_auxi1['Año']=df_auxi1['release_date'].dt.year
df_auxi1=df_auxi1.drop(columns='release_date')
#df para unir con el de arriba
df_auxi2 = df2[['item_id','recommend','sentiment_analysis']]
df_auxi2.rename(columns={'item_id':'id'}, inplace=True)
df_auxi3 = pd.merge(df_auxi1,df_auxi2,on='id')
#limpiar todo el df una vez unido
df_auxi3.dropna(inplace=True)
df_auxi3['Año'] = df_auxi3['Año'].astype(int)
df_auxi3= df_auxi3[(df_auxi3['sentiment_analysis']!=0)&(df_auxi3['sentiment_analysis']!=1)]
df_auxi3= df_auxi3[(df_auxi3['recommend']!=False)]
df_auxi3.drop(columns='id',inplace=True)
df_auxi3=df_auxi3.groupby(['developer','Año', 'recommend'])['sentiment_analysis'].sum().reset_index()
df_auxi3['sentiment_analysis']= (df_auxi3['sentiment_analysis']/2).round().astype(int)
df_auxi4 = df_auxi3[df_auxi3['Año']==ano]
df_auxi4 = df_auxi4.sort_values(by='sentiment_analysis', ascending=False).reset_index(drop=True)
try:
    puesto_1=df_auxi4['developer'].loc[0]
except Exception as e:
    puesto_1 = 'Este puesto esta basio' 
try:
    puesto_2=df_auxi4['developer'].loc[1]
except Exception as e:
    puesto_2 = 'Este puesto esta basio'
try:
    puesto_3=df_auxi4['developer'].loc[2]
except Exception as e:
    puesto_3 = 'Este puesto esta basio'
dicc_salida = [{'Puesto 1':puesto_1},{'Puesto 2':puesto_2},{'Puesto 3':puesto_3}]


C:\Users\gaba_\AppData\Local\Temp\ipykernel_6256\2276199116.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auxi1['id'] = df_auxi1['id'].astype(int)
C:\Users\gaba_\AppData\Local\Temp\ipykernel_6256\2276199116.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auxi1['Año']=df_auxi1['release_date'].dt.year
C:\Users\gaba_\AppData\Local\Temp\ipykernel_6256\2276199116.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [81]:
df_auxi3= df_auxi3[(df_auxi3['sentiment_analysis']!=0)&(df_auxi3['sentiment_analysis']!=1)]
df_auxi3= df_auxi3[(df_auxi3['recommend']!=False)]
df_auxi3.drop(columns='id',inplace=True)
df_auxi3=df_auxi3.groupby(['developer','Año', 'recommend'])['sentiment_analysis'].sum().reset_index()
df_auxi3['sentiment_analysis']= (df_auxi3['sentiment_analysis']/2).round().astype(int)
df_auxi4 = df_auxi3[df_auxi3['Año']==ano]
df_auxi4 = df_auxi4.sort_values(by='sentiment_analysis', ascending=False).reset_index(drop=True)
dicc_salida = [{'Puesto 1':df_auxi4['developer'].loc[0]},{'Puesto 2':df_auxi4['developer'].loc[1]},{'Puesto 3':df_auxi4['developer'].loc[2]}]

In [ ]:
def est_developer_year(ano):
   

    df_auxi1= df4[['release_date','id','developer']]
    df_auxi1['id'] = df_auxi1['id'].astype(int)
    df_auxi1['Año']=df_auxi1['release_date'].dt.year
    df_auxi1=df_auxi1.drop(columns='release_date')
    #df para unir con el de arriba
    df_auxi2 = df2[['item_id','recommend','sentiment_analysis']]
    df_auxi2.rename(columns={'item_id':'id'}, inplace=True)
    df_auxi3 = pd.merge(df_auxi1,df_auxi2,on='id')
    #limpiar todo el df una vez unido
    df_auxi3.dropna(inplace=True)
    df_auxi3['Año'] = df_auxi3['Año'].astype(int)
    df_auxi3= df_auxi3[(df_auxi3['sentiment_analysis']!=0)&(df_auxi3['sentiment_analysis']!=1)]
    df_auxi3= df_auxi3[(df_auxi3['recommend']!=False)]
    df_auxi3.drop(columns='id',inplace=True)
    df_auxi3=df_auxi3.groupby(['developer','Año', 'recommend'])['sentiment_analysis'].sum().reset_index()
    df_auxi3['sentiment_analysis']= (df_auxi3['sentiment_analysis']/2).round().astype(int)
    df_auxi4 = df_auxi3[df_auxi3['Año']==ano]
    df_auxi4 = df_auxi4.sort_values(by='sentiment_analysis', ascending=False).reset_index(drop=True)
    dicc_salida = [{'Puesto 1':df_auxi4['developer'].loc[0]},{'Puesto 2':df_auxi4['developer'].loc[1]},{'Puesto 3':df_auxi4['developer'].loc[2]}]
    return diccionario_salida

In [57]:
def UserForGenre1(genero):
    # Asegurarse de que la fecha esté en formato datetime
    df4['release_date'] = pd.to_datetime(df4['release_date'], errors='coerce')
    
    # Filtrar los juegos que pertenecen al género especificado
    juegos_genero = df4[df4['genres'].str.contains(genero, na=False)]
    
    # Crear un DataFrame con los juegos filtrados y sus años de lanzamiento
    df_juegos = juegos_genero[['id', 'release_date']].copy()
    df_juegos['año'] = df_juegos['release_date'].dt.year
    df_juegos = df_juegos.drop(columns=['release_date']).rename(columns={'id': 'item_id'})
    
    # Filtrar las jugadas correspondientes a los juegos del género
    df_jugadas = df3[df3['item_id'].isin(df_juegos['item_id'])]
    
    # Unir los DataFrames de juegos y jugadas
    new_df = pd.merge(df_juegos, df_jugadas, on='item_id')
    
    # Convertir playtime_forever de minutos a horas, redondear y convertir a entero
    new_df['playtime_forever'] = (new_df['playtime_forever'] / 60).round().astype(int)
    
    # Eliminar la columna item_id
    new_df = new_df.drop(columns=['item_id'])
    
    # Agrupar por usuario y sumar las horas jugadas
    filtro_usuario = new_df.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Ordenar el DataFrame por horas jugadas en forma descendente
    filtro_usuario = filtro_usuario.sort_values(by='playtime_forever', ascending=False).reset_index(drop=True)
    
    # Obtener el usuario con más horas jugadas
    usuario = filtro_usuario.loc[0, 'user_id']
    
    # Agrupar por año y usuario, sumando las horas jugadas
    new_df_agrupado = new_df.groupby(['año', 'user_id'])['playtime_forever'].sum().reset_index()
    
    # Filtrar solo las filas del usuario con más horas jugadas
    new_filtrado = new_df_agrupado[new_df_agrupado['user_id'] == usuario]
    
    # Ordenar el DataFrame por año en forma descendente
    new_filtrado = new_filtrado.sort_values(by='año', ascending=False).reset_index(drop=True)
    
    # Crear una lista con los años y las horas jugadas
    lista_aux = new_filtrado[['año', 'playtime_forever']].rename(columns={'playtime_forever': 'Horas'}).to_dict(orient='records')
    
    # Crear el diccionario de salida
    diccionario_salida = {
        f'Usuario con más horas jugadas para Género {genero}': usuario,
        'Horas jugadas': lista_aux
    }

    return diccionario_salida

In [53]:
def UserForGenre(genero):
    #el genero entra po parametro
    #me aseguro que la fecha este en foramo data time
    df4['release_date'] = pd.to_datetime(df4['release_date'], errors='coerce')
    #listas para guardar los juegos y los años 
    lista_jugos=[]
    lista_ano=[]
    #variable que guarda los nulos
    val= 0
    #recorro el df4 (dataframe) para obtener los balores de que quiero guardar en las listas
    for i ,j , k in zip(df4['genres'],df4['release_date'].dt.year,df4['id']):
        if genero in i: 
            try:
                lista_ano.append(int(j))
                lista_jugos.append(int(k))
            except Exception as e:
                val+=1
    #creo un df (dataFrame) de las listas de juegos y años
    df= pd.DataFrame({'item_id':lista_jugos,'año':lista_ano})
    #creo las listas para guardar los usuarios los juegos y los minutos par cada juego
    lista_usuarios=[]
    lista_itesm=[]
    lista_horas=[]
    #recorro el df3 (dataframe) para obtener los datos de los usuarios, juegos y minutos jugados para cada juego 
    for i,j,k in zip(df3['user_id'],df3['item_id'],df3['playtime_forever']):
        if j in lista_jugos:
            lista_usuarios.append(i)
            lista_itesm.append(j)
            lista_horas.append(k)
    #creo new_df (dataframe) con los datos anteriores
    new_df = pd.DataFrame({'user_id':lista_usuarios,'item_id':lista_itesm,'playtime_forever':lista_horas})
    #uno los dos dataframes previamente creados por la columna item_id (juegos)
    new_df = pd.merge(df, new_df, on='item_id')
    #ordeno el dataframe por minutos jugados de forma decendiente
    new_df=new_df.sort_values(by='playtime_forever', ascending=False)
    #receteo el indice
    new_df.reset_index(drop=True, inplace=True)
    #paso la colimna playtime_forever de minutos a horas
    new_df['playtime_forever'] = new_df['playtime_forever'].apply(lambda x: x / 60)
    #paso la colimna playtime_forever de float a int (de flotante a entero)
    new_df['playtime_forever'] = new_df['playtime_forever'].round().astype(int)
    #Borro la columna item_id
    new_df=new_df.drop(columns=['item_id'])
    #Creo un dataframe que agrupe los usuarios y sumo las horas jugadas  
    filtro_usuario=new_df.groupby('user_id',)[['playtime_forever']].sum().reset_index()
    #Ordeno el dataframe de forma decendiente
    filtro_usuario=filtro_usuario.sort_values(by='playtime_forever', ascending=False)
    #reseteo el indice
    filtro_usuario.reset_index(drop=True, inplace=True)
    #obtengo el usuaro con mas oras jugadas
    usuario=filtro_usuario['user_id'][0]
    #agrupo los dataframe para que mueste cuantas horas tiene cada usuario por año y receteo el indice
    new_df3 =new_df.groupby(['año','user_id']).agg({'playtime_forever': 'sum'}).reset_index()
    #creo un dataframe que tiene solo al usuario con mas horas jugadas 
    new_filtrado=new_df3[new_df3['user_id']==usuario]
    #Ordeno el dataframe por año
    new_filtrado = new_filtrado.sort_values(by='año', ascending=False)
    #reseteo el indice
    new_filtrado.reset_index(drop=True, inplace=True)
    #Creo una lista para guardar los años y las horas jugadas
    lista_aux=[]
    #recorro el dataframe, obtengo el año y oras jugas
    for i,j in zip(new_filtrado['año'],new_filtrado['playtime_forever']):
        dicc_aux={'Año':i,'Horas':j} #Guardo en un diccionario
        lista_aux.append(dicc_aux) #Agrego los datos a la lista
    #Creo el diccionario que deve devolver la funcion
    diccionario_salida={f'Usuario con más horas jugadas para Género {genero}':usuario,
                            'Horas jugadas':lista_aux}

    return diccionario_salida

In [59]:

lista1 = []


In [45]:
df3.head(3)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561198070234207,33,76561198070234207,http://steamcommunity.com/profiles/76561198070...,43110,Metro 2033,0,0
1,76561198070234207,33,76561198070234207,http://steamcommunity.com/profiles/76561198070...,105600,Terraria,15590,0
2,76561198070234207,33,76561198070234207,http://steamcommunity.com/profiles/76561198070...,38830,CrimeCraft GangWars,1,0


In [25]:
#paso todos precios a float, los que son F2P pasan a NaN
df4.loc[:,'price']=pd.to_numeric(df4['price'], errors='coerce')
#paso los NaN a 0
df4.loc[:,'price']=df4['price'].fillna(0)

In [4]:
rsi=0
rno=0
total=0
for i, j in zip(df2['user_id'],df2['recommend']):
    if i == '76561198089393905':
        if j== True:
            rsi+=1
            total+=1
        else:
            rno+=1
            total+=1
porsentaje = int((rsi/total)*100)

In [21]:
total_items=0
for i, j in zip(df3['user_id'],df3['items_count']):
    if i == '76561198070234207':
        total_items= j
    break

In [22]:
total_items

33

In [10]:
porsentaje = int((rsi/total)*100)
porsentaje

100

In [46]:
lista1=[]
#Filtra por usuario
for i,j in zip(df3['user_id'],df3['item_id']) :
    if i == '76561198134938809':
        lista1.append(j)

In [47]:
lista1

[]

In [59]:
def user_data(user_id):
    lista=[]
    #Filtra por usuario
    for i,j in zip(df3['user_id'],df3['item_id']) :
        if i == user_id:
            lista.append(j)
    #Obtiene el presio de los juegos que tenga y los suma
    precios= 0
    for i, j in zip(df4['id'],df4['price']):
        if i in lista:
            precios+=j 
    dinero_gastado = f'{int(precios)} USD'
    #Obtiene la contidad de recomendaciones
    rsi=0
    rno=0
    total=0
    for i, j in zip(df2['user_id'],df2['recommend']):
        if i == user_id:
            if j== True:
                rsi+=1
                total+=1
            else:
                rno+=1
                total+=1
    if total > 0:
        porsentaje = int((rsi/total)*100)
    else:
        porsentaje = 0
    porsentaje = f'{porsentaje}%'
    #Obtiene la cantidad de items
    total_items=0
    for i, j in zip(df3['user_id'],df3['items_count']):
        if i == user_id:
            total_items= j
        if total_items > 0:
            break
    #retorno de la funcion en formato diccionario
    dic1 ={'usuario': user_id,
           'Dinero gastado': dinero_gastado,
           'Porsentaje de recomendacion':porsentaje,
           'Cantidad de items': total_items
           }
    return dic1

In [65]:
pepe= user_data('lilim')
pepe

{'usuario': 'lilim',
 'Dinero gastado': '0 USD',
 'Porsentaje de recomendacion': '33%',
 'Cantidad de items': 0}

In [ ]:
ruta= 'data comprimida/user_reviews.json.gz'
steam_list=[] # lista de dattos

with gzip.open(ruta,'rt',encoding='utf-8') as archivo:
    for fila in archivo:
        datos=json.loads(fila)
        steam_list.append(datos)

In [ ]:
import pandas as pd

# Leer el archivo JSON modificado
data = pd.read_json("datasets/nuevo.json")

# Si quieres acceder a los items de cada usuario
items = data['items'].explode().apply(pd.Series)

print(data)
print(items)


In [30]:
import json

def formatear_json(archivo_entrada, archivo_salida):
    """
    Formatea un archivo JSON para que pueda ser leído correctamente por pandas.
    
    :param archivo_entrada: Ruta al archivo JSON de entrada.
    :param archivo_salida: Ruta al archivo JSON de salida.
    """
    try:
        # Leer el contenido del archivo JSON de entrada
        with open(archivo_entrada, 'r', encoding='utf-8') as file:
            contenido = json.load(file)
        
        # Formatear el contenido adecuadamente
        datos_formateados = []
        for usuario in contenido:
            usuario_data = {
                "user_id": usuario.get("user_id", ""),
                "items_count": usuario.get("items_count", 0),
                "steam_id": usuario.get("steam_id", ""),
                "user_url": usuario.get("user_url", ""),
                "items": usuario.get("items", [])
            }
            datos_formateados.append(usuario_data)
        
        # Guardar el contenido formateado en el archivo de salida
        with open(archivo_salida, 'w', encoding='utf-8') as file:
            json.dump(datos_formateados, file, indent=4)
        
        print(f"Archivo formateado guardado en {archivo_salida}")
    
    except json.JSONDecodeError as e:
        print(f"Error al leer el archivo JSON: {e}")
    except Exception as e:
        print(f"Se produjo un error: {e}")

# Usar la función para formatear el archivo proporcionado
archivo_entrada = 'datasets/nuevo.json'  # Cambia esta ruta por la ruta de tu archivo
archivo_salida = 'datasets/nuevo_modificado.json'  # Cambia esta ruta por donde quieres guardar el archivo modificado

formatear_json(archivo_entrada, archivo_salida)


Error al leer el archivo JSON: Expecting ',' delimiter: line 1 column 1394 (char 1393)


In [4]:
import re
import json

# Ruta del archivo JSON original
file_path = 'datasets/australian_users_items.json'

# Leer el archivo con comillas simples
with open(file_path, 'r', encoding='utf-8') as file:
    json_content = file.read()

# Expresión regular para reemplazar comillas simples por comillas dobles en las claves y valores del JSON
json_corrected = re.sub(r"(?<!\\)'", '"', json_content)

# Guardar el JSON corregido en un nuevo archivo
corrected_file_path = 'datasets/australian_users_items_corrected.json'
with open(corrected_file_path, 'w', encoding='utf-8') as file:
    file.write(json_corrected)

print(f"Archivo JSON corregido guardado en: {corrected_file_path}")

Archivo JSON corregido guardado en: datasets/australian_users_items_corrected.json


In [ ]:
import re
import json

# Ruta del archivo JSON original
file_path = 'datasets/australian_users_items.json'

# Leer el archivo con comillas simples
with open(file_path, 'r', encoding='utf-8') as file:
    json_content = file.read()

# Expresión regular para reemplazar comillas simples por comillas dobles en las claves y valores del JSON
json_corrected = re.sub(r"(?<!\\)'", '"', json_content)

# Guardar el JSON corregido en un nuevo archivo
corrected_file_path = 'datasets/australian_user_reviews_corrected.json'
with open(corrected_file_path, 'w', encoding='utf-8') as file:
    file.write(json_corrected)

print(f"Archivo JSON corregido guardado en: {corrected_file_path}")

# Cargar el archivo JSON corregido
with open(corrected_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Transformar la estructura
reviews = []
for user in data:
    user_id = user.get('user_id')
    user_url = user.get('user_url')
    for review in user.get('reviews', []):
        review_entry = {
            'user_id': user_id,
            'user_url': user_url,
            'item_id': review.get('item_id'),
            'recommend': review.get('recommend'),
            'review': review.get('review'),
            'funny': review.get('funny'),
            'helpful': review.get('helpful'),
            'posted': review.get('posted'),
            'last_edited': review.get('last_edited'),
        }
        reviews.append(review_entry)

# Guardar el nuevo JSON transformado
transformed_file_path = 'transformed_reviews.json'
with open(transformed_file_path, 'w', encoding='utf-8') as file:
    json.dump(reviews, file, ensure_ascii=False, indent=4)

print(f"Transformación completada y guardada en: {transformed_file_path}")


In [7]:
import ast
import json

# Ruta del archivo JSON original
file_path = 'datasets/australian_user_reviews - copia.json'

# Leer el archivo con comillas simples
with open(file_path, 'r', encoding='utf-8') as file:
    json_content = file.read()

# Usar ast.literal_eval para corregir las comillas
try:
    corrected_json_content = ast.literal_eval(json_content)
except Exception as e:
    print(f"Error al evaluar el contenido JSON: {e}")
    corrected_json_content = None

# Guardar el JSON corregido en un nuevo archivo si no hubo errores
if corrected_json_content is not None:
    corrected_file_path = 'datasets/australian_user_reviews_corrected.json'
    with open(corrected_file_path, 'w', encoding='utf-8') as file:
        json.dump(corrected_json_content, file, ensure_ascii=False, indent=4)
    print(f"Archivo JSON corregido guardado en: {corrected_file_path}")

    # Cargar el archivo JSON corregido
    with open(corrected_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Transformar la estructura
    reviews = []
    for user in data:
        user_id = user.get('user_id')
        user_url = user.get('user_url')
        for review in user.get('reviews', []):
            review_entry = {
                'user_id': user_id,
                'user_url': user_url,
                'item_id': review.get('item_id'),
                'recommend': review.get('recommend'),
                'review': review.get('review'),
                'funny': review.get('funny'),
                'helpful': review.get('helpful'),
                'posted': review.get('posted'),
                'last_edited': review.get('last_edited'),
            }
            reviews.append(review_entry)

    # Guardar el nuevo JSON transformado
    transformed_file_path = 'transformed_reviews.json'
    with open(transformed_file_path, 'w', encoding='utf-8') as file:
        json.dump(reviews, file, ensure_ascii=False, indent=4)

    print(f"Transformación completada y guardada en: {transformed_file_path}")

else:
    print("No se pudo corregir el JSON.")


Error al evaluar el contenido JSON: invalid syntax (<unknown>, line 2)
No se pudo corregir el JSON.


In [ ]:
import json

# Cargar el archivo JSON original
with open('australian_user_reviews', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Transformar la estructura
reviews = []
for user in data:
    user_id = user.get('user_id')
    user_url = user.get('user_url')
    for review in user.get('reviews', []):
        review_entry = {
            'user_id': user_id,
            'user_url': user_url,
            'item_id': review.get('item_id'),
            'recommend': review.get('recommend'),
            'review': review.get('review'),
            'funny': review.get('funny'),
            'helpful': review.get('helpful'),
            'posted': review.get('posted'),
            'last_edited': review.get('last_edited'),
        }
        reviews.append(review_entry)

# Guardar el nuevo JSON
with open('transformed_reviews.json', 'w', encoding='utf-8') as file:
    json.dump(reviews, file, ensure_ascii=False, indent=4)
